In [ ]:
!pip install numpy
!pip install pandas
!pip install sklearn
!pip install seaborn
!pip install matplotlib

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Carico i dati e ricostruisco per comodità una tabella unica, etichettando le righe che saranno di train e quelle che saranno di test

In [3]:
train_set = pd.read_csv("train.csv")
test_set = pd.read_csv("test.csv")

train_set["test"] = 0
test_set["test"] = 1

all_data = pd.concat([train_set, test_set])

Stampo alcune prime informazioni sulla natura dei dati

In [ ]:
all_data.info()

In [ ]:
all_data

Come si può notare dalla tabella sotto, CustomerID è un valore univoco per ogni customer; non è pertanto rilevante ai fini dell'analisi e può essere rimosso subito

In [ ]:
all_data.describe(include=["object"])

In [7]:
all_data = all_data.drop(labels=["customerID"], axis=1)

In [8]:
def get_uniques(df, columns):
  return {column: list(df[column].unique()) for column in columns}

def get_object_columns(df):
  return [column for column in df.columns if df.dtypes[column] == "object"]

def get_binaries(df):
  obj_cols = get_uniques(df, get_object_columns(df))
  return [column for column in obj_col if len(obj_cols[column])==2]

La colonna "TotalCharge" ha valori che rappresentano float identificati come stringhe. 

In [9]:
all_data["TotalCharges"] = all_data["TotalCharges"].replace(" ", np.NaN)
all_data["TotalCharges"] = all_data["TotalCharges"].astype(np.float)
all_data["TotalCharges"] = all_data["TotalCharges"].fillna(all_data["TotalCharges"].mean())

In [ ]:
obj_col = get_uniques(all_data, get_object_columns(all_data))
obj_col

Effettuiamo un encoding dei valori delle features

In [ ]:
toChange = [column for column in obj_col if column != "Contract" and column !="InternetService" and len(obj_col[column])==3]
toChange

In [12]:
for column in toChange:
  all_data[column] = all_data[column].apply(lambda x: x if x=="Yes" else "No")

In [ ]:
get_uniques(all_data, get_object_columns(all_data))

In [ ]:
get_binaries(all_data)

In [15]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
for column in get_binaries(all_data):
  all_data[column] = encoder.fit_transform(all_data[column])

In [16]:
# OneHotEncoding using panda get_dummies


for col in get_object_columns(all_data):
    dummies = pd.get_dummies(all_data[col])
    all_data = pd.concat([all_data, dummies], axis=1)
    all_data = all_data.drop(col, axis=1)


In [ ]:
all_data

##Plotting correlations

In [ ]:
plt.figure(figsize=(25,10))
sns.heatmap(all_data.corr(), annot=True, linewidths=2)

Scelgo di droppare "gender" e "PhoneService" poiché hanno una correlazione davvero minima con la colonna desiderata.
Droppo anche "TotalCharges" in quanto ha elevata correlazione con molte altre features ed è quindi facilmente deducibile dalle altre.

In [24]:
toDrop = ["gender", "PhoneService", "TotalCharges"]

all_data = all_data.drop(toDrop, axis=1)

In [ ]:
all_data.head(10)

### Riseparazione dei dati


In [26]:
train_set = all_data[all_data.test == 0].drop("test", axis=1)
test_set = all_data[all_data.test == 1].drop("test", axis=1)

In [27]:
y_train = train_set.Churn
x_train = train_set.drop("Churn", axis=1)
y_test = test_set.Churn
x_test = test_set.drop("Churn", axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, plot_confusion_matrix, classification_report

model = LogisticRegression(random_state = 0, max_iter = 1000)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print(f"Acc: {accuracy_score(y_pred, y_test)}")
plot_confusion_matrix(model, x_test, y_test, values_format='n', display_labels=['No','Yes'])
plt.show()
print(classification_report(y_test,y_pred))
